In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import *
from pyspark.sql import Window

sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("FastFood_Batch_Analytics")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the Spark session, which is the entry point to the Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Read the fast food ordering dataset
gsc_file_path = 'gs://data_de2025_2062061/fast_food_ordering_dataset.csv'

df = spark.read.format("csv").option("header", "true") \
    .load(gsc_file_path)

# Data type conversions
df = df.withColumn("order_value", col("order_value").cast("double")) \
       .withColumn("delivery_time_minutes", col("delivery_time_minutes").cast("integer")) \
       .withColumn("items_count", col("items_count").cast("integer")) \
       .withColumn("order_time", to_timestamp(col("order_time"), "yyyy-MM-dd HH:mm:ss"))

df.printSchema()
df.show(10)


### Transformation 1: Revenue Analysis by City

Calculate total revenue, average order value, total orders, and average delivery time for each city.


In [ ]:
# Revenue Analysis by City
revenue_by_city = df.groupBy("city") \
    .agg(
        sum("order_value").alias("total_revenue"),
        avg("order_value").alias("avg_order_value"),
        count("*").alias("total_orders"),
        avg("delivery_time_minutes").alias("avg_delivery_time")
    ) \
    .orderBy(desc("total_revenue"))

revenue_by_city.show()


### Transformation 2: Cuisine Type Performance

Analyze performance metrics for each cuisine type including order count, revenue, and delivery times.


In [ ]:
# Cuisine Type Performance
cuisine_performance = df.groupBy("cuisine_type") \
    .agg(
        count("*").alias("order_count"),
        sum("order_value").alias("total_revenue"),
        avg("order_value").alias("avg_order_value"),
        avg("items_count").alias("avg_items_per_order"),
        avg("delivery_time_minutes").alias("avg_delivery_time")
    ) \
    .orderBy(desc("total_revenue"))

cuisine_performance.show()


### Transformation 3: Payment Method Analysis

Analyze payment method usage patterns and revenue contribution.


In [ ]:
# Payment Method Analysis
payment_analysis = df.groupBy("payment_method") \
    .agg(
        count("*").alias("usage_count"),
        sum("order_value").alias("total_revenue"),
        avg("order_value").alias("avg_order_value")
    ) \
    .orderBy(desc("usage_count"))

payment_analysis.show()


### Transformation 4: Top Performing Cities by Cuisine (Window Functions)

Use window functions to rank cities by revenue within each cuisine type and identify top 3 performers.


In [ ]:
# Top Performing Cities by Cuisine (Window Functions)
window_spec = Window.partitionBy("cuisine_type").orderBy(desc("total_revenue"))

city_cuisine_revenue = df.groupBy("city", "cuisine_type") \
    .agg(sum("order_value").alias("total_revenue")) \
    .withColumn("rank", dense_rank().over(window_spec)) \
    .filter(col("rank") <= 3) \
    .orderBy("cuisine_type", "rank")

city_cuisine_revenue.show(50)


### Transformation 5: Daily Revenue Trends

Analyze revenue trends over time by aggregating orders by date.


In [ ]:
# Daily Revenue Trends
daily_revenue = df.withColumn("order_date", to_date(col("order_time"))) \
    .groupBy("order_date") \
    .agg(
        sum("order_value").alias("daily_revenue"),
        count("*").alias("daily_orders"),
        avg("order_value").alias("avg_order_value")
    ) \
    .orderBy("order_date")

daily_revenue.show(30)


### Transformation 6: High-Value Orders Analysis

Identify top 10% of orders by value using percentile ranking.


In [ ]:
# High-Value Orders Analysis (Top 10%)
window_high_value = Window.orderBy(desc("order_value"))
high_value_orders = df.withColumn("percent_rank", percent_rank().over(window_high_value)) \
    .filter(col("percent_rank") <= 0.1) \
    .select("order_id", "city", "cuisine_type", "order_value", "items_count", "payment_method")

high_value_orders.show(20)


### Transformation 7: Delivery Performance by City

Analyze delivery efficiency metrics and categorize cities by delivery speed.


In [ ]:
# Delivery Performance by City
delivery_performance = df.groupBy("city") \
    .agg(
        avg("delivery_time_minutes").alias("avg_delivery_time"),
        min("delivery_time_minutes").alias("min_delivery_time"),
        max("delivery_time_minutes").alias("max_delivery_time"),
        count("*").alias("total_orders")
    ) \
    .withColumn("delivery_efficiency", 
                when(col("avg_delivery_time") < 35, "Fast")
                .when(col("avg_delivery_time") < 50, "Medium")
                .otherwise("Slow")) \
    .orderBy("avg_delivery_time")

delivery_performance.show()


In [ ]:
# Setup Hadoop FS configuration for schema gs://
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

# Use the Cloud Storage bucket for temporary BigQuery export data that the connector uses.
bucket = "temp_de2025_2062061"  
spark.conf.set('temporaryGcsBucket', bucket)

# Save revenue_by_city to BigQuery
revenue_by_city.write.format('bigquery') \
  .option('table', 'de2025-472319.labdataset.revenue_by_city') \
  .mode("overwrite") \
  .save()

# Save cuisine_performance to BigQuery
cuisine_performance.write.format('bigquery') \
  .option('table', 'de2025-472319.labdataset.cuisine_performance') \
  .mode("overwrite") \
  .save()


In [ ]:
# Stop the Spark context
spark.stop()
